In this notebook, few regression types are tried using sklearn. They are trained on dataset X_train_observed and then crossvalidated on X_train_estimated (there is no particular reason behind dividing the dataset to training and testing part like this, I just decided it would be easiest way). 

This crossvalidation is tried just on dataset A. Then the solution with the lowest mean absolute error (mae) is chosen and used for the other datasets.


In [1]:
%load_ext autoreload

In [2]:
%autoreload

# load libraries
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR
import h2o
from h2o.automl import H2OAutoML

# load my custom functions
from solutions.few_regression_types import data_preprocess


In [3]:
# read dataset A
# for simplicity, I use X_train_estimated as test data for cross validation
y = pd.read_parquet("../../dataset/A/train_targets.parquet")
X_train = pd.read_parquet("../../dataset/A/X_train_observed.parquet")
X_test = pd.read_parquet("../../dataset/A/X_train_estimated.parquet")

In [4]:
# edit data
X_train, y_train = data_preprocess.preprocess_train_data(X_train, y, "everything")
X_test, y_test = data_preprocess.preprocess_train_data(X_test, y, "everything")

In [5]:
print(f"X_train.shape = {X_train.shape}")
print(f"X_test.shape = {X_test.shape}")
print(f"y_train.shape = {y_train.shape}")
print(f"y_test.shape = {y_test.shape}")

X_train.shape = (29667, 47)
X_test.shape = (4394, 47)
y_train.shape = (29667, 1)
y_test.shape = (4394, 1)


## Machine learning methods

In [6]:
# decision tree
decision_tree = DecisionTreeRegressor()
decision_tree.fit(X_train, y_train)
y_pred_tree = decision_tree.predict(X_test)

mae_tree = np.mean(np.abs(np.array(y_test) - y_pred_tree))
mae_tree    # mae means mean absolute error, mae_tree = 616.575890061115

622.6471723254433

In [7]:
# random forest
random_forest = RandomForestRegressor(n_estimators=100)
random_forest.fit(X_train, y_train.values.ravel()) # ravel part is because of scikit's data conversion warning, it does not have to be there
y_pred_forest = random_forest.predict(X_test)

mae_forest = np.mean(np.abs(np.array(y_test) - y_pred_forest))
mae_forest  # 599.9553060312836

600.4668224222045

In [8]:
# gradient boosting
gradient_boosting = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1) 
gradient_boosting.fit(X_train, y_train.values.ravel())
y_pred_grad = gradient_boosting.predict(X_test)

mae_grad = np.mean(np.abs(np.array(y_test) - y_pred_grad))
mae_grad    # 592.2928322998536

592.2928322998534

In [9]:
# elastic net
elastic_net = ElasticNet(alpha=1.0, l1_ratio=0.5, random_state=42)
elastic_net.fit(X_train, y_train.values.ravel())
y_pred_elast_net = elastic_net.predict(X_test)

mae_elast_net = np.mean(np.abs(np.array(y_test) - y_pred_elast_net))
mae_elast_net   # 599.946498024572

C:\Users\simon\OneDrive\Dokumenty\Solar-Energy-Production-Forecasting\venv\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.684e+09, tolerance: 4.240e+06
  model = cd_fast.enet_coordinate_descent(


599.9464980245721

In [10]:
# support vector regression
svr_model = SVR(kernel='rbf', C=1.)
svr_model.fit(X_train, y_train.values.ravel())
y_pred_svr = svr_model.predict(X_test)

mae_svr = np.mean(np.abs(np.array(y_test) - y_pred_svr))
mae_svr

473.76274242555763

In [11]:
# hyperparameter tuning of SVR
# no need to run this section, it takes too long; the results are approx.: 
# [345, 362, 397, 437, 473, 497, 513] 
# lower C gives us better results
for C in [0.001, 0.03, 0.1, 0.3, 1, 3, 10]:
    svr_model = SVR(kernel='rbf', C=C)
    svr_model.fit(X_train, y_train.values.ravel())
    y_pred_svr = svr_model.predict(X_test)

    print(np.mean(np.abs(np.array(y_test) - y_pred_svr)), end=", ")

345.1870758178765, 362.3174570223702, 397.974746466719, 437.0422504854231, 473.76274242555763, 497.00397971674676, 513.0037055004162, 

In [12]:
#H2O AutoML

y = pd.read_parquet("../../dataset/A/train_targets.parquet")
X_train = pd.read_parquet("../../dataset/A/X_train_observed.parquet")
X_test = pd.read_parquet("../../dataset/A/X_train_estimated.parquet")

X_train, y_train = data_preprocess.preprocess_train_data(X_train, y, "everything")
X_test, y_test = data_preprocess.preprocess_train_data(X_test, y, "everything")

h2o.init(max_mem_size = "16G") # If u have more RAM change the parameter

train_frame = h2o.H2OFrame(pd.concat([X_train, y_train], axis=1))
test_frame = h2o.H2OFrame(pd.concat([X_test, y_test], axis=1))

x = train_frame.columns[:-1] 
y = train_frame.columns[-1] 

aml = H2OAutoML(max_runtime_secs=60,max_models = 5)
aml.train(x=x, y=y, training_frame=train_frame, validation_frame=test_frame)

print(aml.leaderboard)
best_model = aml.leader

h2o.shutdown()


Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,7 mins 25 secs
H2O_cluster_timezone:,Europe/Prague
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.1
H2O_cluster_version_age:,"21 days, 4 hours and 22 minutes"
H2O_cluster_name:,H2O_from_python_simon_ac7n4x
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,13.88 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |█
14:49:07.466: AutoML: XGBoost is not available; skipping it.
14:49:07.506: _train param, Dropping bad and constant columns: [snow_drift:idx, elevation:m]
14:49:08.415: _train param, Dropping bad and constant columns: [snow_drift:idx, elevation:m]
14:49:10.626: _train param, Dropping bad and constant columns: [snow_drift:idx, elevation:m]

██████████
14:49:21.957: _train param, Dropping bad and constant columns: [snow_drift:idx, elevation:m]
14:49:22.861: _train param, Dropping bad and constant columns: [snow_drift:idx, elevation:m]
14:49:24.68: _train param, Dropping unused columns: [snow_drift:idx, elevation:m]
14:49:24.681: _train param, Dropping unused columns: [snow_drift:idx, elevation:m]

████████████████████████████████████████████████████| (done) 100%
model_id            

C:\Users\simon\AppData\Local\Temp\ipykernel_7100\72466135.py:24: H2ODeprecationWarning: Deprecated, use ``h2o.cluster().shutdown()``.
  h2o.shutdown()


H2O session _sid_b502 closed.


# Prediction on real test data


SVR model came out with the lowest mean absolute error. So far we did only cross validation on the training data. SVR will be used on the real test data, on the datasets B and C and to generate the output csv file.

In [ ]:
prediction = []

for letter in ['A', 'B', 'C']:
    # read the data
    print(f"dataset {letter}")
    X_train = pd.concat([
        pd.read_parquet(f"../../dataset/{letter}/X_train_observed.parquet"),
        pd.read_parquet(f"../../dataset/{letter}/X_train_estimated.parquet")
    ], ignore_index=True)
    y_train = pd.read_parquet(f"../../dataset/{letter}/train_targets.parquet")
    X_test = pd.read_parquet(f"../../dataset/{letter}/X_test_estimated.parquet")
    # preprocess the data
    X_train, y_train = data_preprocess.preprocess_train_data(X_train, y_train, "everything")
    X_test = data_preprocess.preprocess_test_data(X_test, "everything")
    # learn 
    model = SVR(kernel='rbf', C=.001)
    model.fit(X_train, y_train.values.ravel())
    prediction = np.concatenate((prediction, model.predict(X_test)))
prediction[prediction < 0.] = 0. # energy production can't be negative
df = pd.DataFrame({'prediction': prediction})
df.to_csv('svr.csv', index_label='id')
print("done")

In [21]:
prediction = []
for letter in ['A', 'B', 'C']:
    # read the data
    print(f"dataset {letter}")
    X_train = pd.concat([
        pd.read_parquet(f"../../dataset/{letter}/X_train_observed.parquet"),
        pd.read_parquet(f"../../dataset/{letter}/X_train_estimated.parquet")
    ], ignore_index=True)
    y_train = pd.read_parquet(f"../../dataset/{letter}/train_targets.parquet")
    X_test = pd.read_parquet(f"../../dataset/{letter}/X_test_estimated.parquet")
    # preprocess the data
    X_train, y_train = data_preprocess.preprocess_train_data(X_train, y_train, "everything")
    X_test = data_preprocess.preprocess_test_data(X_test, "everything")
    
    h2o.init(max_mem_size = "16G") # If u have more RAM change the parameter

    train_frame = h2o.H2OFrame(pd.concat([X_train, y_train], axis=1))
    test_frame = h2o.H2OFrame(pd.concat([X_test, y_test], axis=1))

    x = train_frame.columns[:-1] 
    y = train_frame.columns[-1] 

    model1 = H2OAutoML(max_runtime_secs=60)
    model1.train(x=x, y=y, training_frame=train_frame, validation_frame=test_frame)

    print(model1.leaderboard)
    prediction = best_model.predict(test_frame)#,seed = 123456) # idk why, but the seed doesnt work for me

    predictions_df = h2o.as_list(prediction)
    
    predictions_df['prediction'][predictions_df['prediction'] < 0] = 0
    
    predictions_df.to_csv('automl.csv', index_label='id')

    print("done")
    h2o.shutdown()

SyntaxError: invalid syntax (1029579131.py, line 15)